# Accessing Art Museums

In [261]:
print('Hello World')

Hello World


In [262]:
#import libraries
import json
import pandas as pd
import numpy as np

# Load data
The following data sets are used in this project:
1. US Art Museums (CSV) - Art museums in the United States with website, city, state, ZIP Code, and cost of admission
2. 2022 Census Income Data (CSV) - Income data by ZIP Code

In [263]:
# read US Art Museums CSV
museums = pd.read_csv('data/US_Art_Museums.csv')

# display first 5 rows
museums.head()

,Unnamed: 0,Museum,Website,City,State,Zip Code,General Admission,Senior,Student,Children
0,1,Birmingham Museum of Art,artsbma.org,Birmingham,Alabama,35203,$0.00,$0.00,$0.00,$0.00
1,2,Huntsville Museum of Art,hsvmuseum.org,Huntsville,Alabama,35801,$10.00,$8.00,$5.00,$0.00
2,3,Mobile Museum of Art,mobilemuseumofart.com,Mobile,Alabama,36608,$12.00,$10.00,$8.00,$0.00
3,4,Montgomery Museum of Fine Arts,mmfa.org,Montgomery,Alabama,36117,$0.00,$0.00,$0.00,$0.00
4,5,Anchorage Museum of Art and History,anchoragemuseum.org,Anchorage,Alaska,99501,$18.00,$15.00,$9.00,$0.00


In [264]:
# read 2022 Census Income Data CSV
income = pd.read_csv('data/2022_Census_Income_Data.csv')

# display first 5 rows
income.head()

# LEADING ZERO PROBLEM

,Geography,Geographic area name,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!White,Margin of Error!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!White,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!Black or African American,Margin of Error!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!Black or African American,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!American Indian and Alaska Native,Margin of Error!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!American Indian and Alaska Native,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!Asian,Margin of Error!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!Asian,...,Estimate!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Female householder!!Living alone,Margin of Error!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Female householder!!Living alone,Estimate!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Female householder!!Not living alone,Margin of Error!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Female householder!!Not living alone,Estimate!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Male householder,Margin of Error!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Male householder,Estimate!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Male householder!!Living alone,Margin of Error!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Male householder!!Living alone,Estimate!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Male householder!!Not living alone,Margin of Error!!Median income (dollars)!!NONFAMILY HOUSEHOLDS!!Nonfamily households!!Male householder!!Not living alone
0,860Z200US00601,601,4564,334,64,42,4,8,20,32,...,9895,1455,21154,5589,11208,2272,9403,3371,23352,4538
1,860Z200US00602,602,6042,431,135,81,15,22,0,31,...,8669,1008,23676,17821,16086,1422,15352,1651,28872,17030
2,860Z200US00603,603,14117,695,865,183,20,23,19,29,...,9229,954,16333,1539,13147,1542,12382,1635,28824,12175
3,860Z200US00606,606,1185,163,0,21,0,21,0,21,...,11938,7641,-,**,17899,4102,17269,6376,18850,8182
4,860Z200US00610,610,3548,443,189,95,0,28,0,28,...,11913,1706,21406,5972,15247,2151,13688,3051,25658,7664


# Clean data

### Census Data CSV
The Census Data CSV includes a wide range of income measurements for specific races, genders, and household types, as well as margins of error for each measurement. For the purpose of this project, we are looking for the median income of all types calculated as one amount, so the data will need to be narrowed down and most columns will be removed.


In [265]:
# identify all columns
print(income.columns)

Index(['Geography', 'Geographic area name',
       'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!White',
       'Margin of Error!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!White',
       'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!Black or African American',
       'Margin of Error!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!Black or African American',
       'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!American Indian and Alaska Native',
       'Margin of Error!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!American Indian and Alaska Native',
       'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC O

In [266]:
# fiter down to necessary columns
income = income[['Geographic area name', 'Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households']]

# display remaining columns of data frame
income.head()

,Geographic area name,Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households
0,601,17526.0
1,602,20260.0
2,603,17703.0
3,606,19603.0
4,610,22796.0


In [267]:
# rename columns
income.rename(columns={'Geographic area name': 'Zip Code', 'Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households': 'Median Income'}, inplace=True)

# display new columns
income.head()

,Zip Code,Median Income
0,601,17526.0
1,602,20260.0
2,603,17703.0
3,606,19603.0
4,610,22796.0


The Census Data CSV includes more zip codes than are needed for comparing with the Art Museums CSV. Before merging, we will remove uneeded zip codes from the data frame.

In [268]:
# remove zip codes from Census Data that are not found in Art Museums
income = income[income['Zip Code'].isin(museums['Zip Code'])]

#check new number of zip codes in Census Data
income.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 53 to 33508
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Zip Code       300 non-null    int64  
 1   Median Income  273 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 7.0 KB


### Art Museums CSV
The Art Museums CSV includes data for specific groups of visitors. For the purpose of this project, we are looking at General Admission prices so the data for other groups can be removed.

In [269]:
# fiter down to necessary columns
museums = museums[['Unnamed: 0', 'Museum', 'Website', 'City', 'State', 'Zip Code', 'General Admission']]

# display remaining columns of data frame
museums.head()

,Unnamed: 0,Museum,Website,City,State,Zip Code,General Admission
0,1,Birmingham Museum of Art,artsbma.org,Birmingham,Alabama,35203,$0.00
1,2,Huntsville Museum of Art,hsvmuseum.org,Huntsville,Alabama,35801,$10.00
2,3,Mobile Museum of Art,mobilemuseumofart.com,Mobile,Alabama,36608,$12.00
3,4,Montgomery Museum of Fine Arts,mmfa.org,Montgomery,Alabama,36117,$0.00
4,5,Anchorage Museum of Art and History,anchoragemuseum.org,Anchorage,Alaska,99501,$18.00


# Merge data

In [270]:
df = museums.merge(income, on='Zip Code')
df

,Unnamed: 0,Museum,Website,City,State,Zip Code,General Admission,Median Income
0,1,Birmingham Museum of Art,artsbma.org,Birmingham,Alabama,35203,$0.00,42596.0
1,2,Huntsville Museum of Art,hsvmuseum.org,Huntsville,Alabama,35801,$10.00,97643.0
2,3,Mobile Museum of Art,mobilemuseumofart.com,Mobile,Alabama,36608,$12.00,59400.0
3,4,Montgomery Museum of Fine Arts,mmfa.org,Montgomery,Alabama,36117,$0.00,73919.0
4,5,Anchorage Museum of Art and History,anchoragemuseum.org,Anchorage,Alaska,99501,$18.00,73540.0
...,...,...,...,...,...,...,...,...
351,361,Milwaukee Art Museum,mam.org,Milwaukee,Wisconsin,53202,$19.00,62505.0
352,362,Villa Terrace Decorative Arts Museum,villaterrace.org,Milwaukee,Wisconsin,53211,$10.00,73783.0
353,363,Museum of Wisconsin Art,wisconsinart.org,West Bend,Wisconsin,53095,$15.00,77397.0
354,364,National Museum of Wildlife Art,wildlifeart.org,Jackson Hole,Wyoming,83001,$20.00,110734.0


In [271]:
# rename Unnamed: 0 to ID
df.rename(columns={'Unnamed: 0':'ID'}, inplace=True )

# rename Zip Code to ZIP code
df.rename(columns={'Zip Code':'ZIP Code'}, inplace=True )
df.head()

,ID,Museum,Website,City,State,ZIP Code,General Admission,Median Income
0,1,Birmingham Museum of Art,artsbma.org,Birmingham,Alabama,35203,$0.00,42596.0
1,2,Huntsville Museum of Art,hsvmuseum.org,Huntsville,Alabama,35801,$10.00,97643.0
2,3,Mobile Museum of Art,mobilemuseumofart.com,Mobile,Alabama,36608,$12.00,59400.0
3,4,Montgomery Museum of Fine Arts,mmfa.org,Montgomery,Alabama,36117,$0.00,73919.0
4,5,Anchorage Museum of Art and History,anchoragemuseum.org,Anchorage,Alaska,99501,$18.00,73540.0


Clean merged data

In [272]:
# print data types to see which strings need to be converted to integers
print(df)
print(df.dtypes)

      ID                                Museum                Website  \
0      1              Birmingham Museum of Art            artsbma.org   
1      2              Huntsville Museum of Art          hsvmuseum.org   
2      3                  Mobile Museum of Art  mobilemuseumofart.com   
3      4        Montgomery Museum of Fine Arts               mmfa.org   
4      5   Anchorage Museum of Art and History    anchoragemuseum.org   
..   ...                                   ...                    ...   
351  361                  Milwaukee Art Museum                mam.org   
352  362  Villa Terrace Decorative Arts Museum       villaterrace.org   
353  363               Museum of Wisconsin Art       wisconsinart.org   
354  364       National Museum of Wildlife Art        wildlifeart.org   
355  365                    The Brinton Museum   thebrintonmuseum.org   

                City      State  ZIP Code General Admission  Median Income  
0         Birmingham    Alabama     35203     

In [273]:
# remove all NaN from Median Income
df.dropna(subset=["Median Income"], inplace=True)

# convert Median Income column from string to integer
df['Median Income'] = df['Median Income'].astype(float)
print(df)
print(df.dtypes)


      ID                                       Museum                 Website  \
0      1                     Birmingham Museum of Art             artsbma.org   
1      2                     Huntsville Museum of Art           hsvmuseum.org   
2      3                         Mobile Museum of Art   mobilemuseumofart.com   
3      4               Montgomery Museum of Fine Arts                mmfa.org   
4      5          Anchorage Museum of Art and History     anchoragemuseum.org   
..   ...                                          ...                     ...   
350  360  Patrick and Beatrice Haggerty Museum of Art  marquette.edu/haggerty   
351  361                         Milwaukee Art Museum                 mam.org   
352  362         Villa Terrace Decorative Arts Museum        villaterrace.org   
353  363                      Museum of Wisconsin Art        wisconsinart.org   
354  364              National Museum of Wildlife Art         wildlifeart.org   

             City      Stat

In [274]:
df["General Admission"] = df["General Admission"].replace("[$,]", "", regex=True).astype(float)
print(df)
print(df.dtypes)

      ID                                       Museum                 Website  \
0      1                     Birmingham Museum of Art             artsbma.org   
1      2                     Huntsville Museum of Art           hsvmuseum.org   
2      3                         Mobile Museum of Art   mobilemuseumofart.com   
3      4               Montgomery Museum of Fine Arts                mmfa.org   
4      5          Anchorage Museum of Art and History     anchoragemuseum.org   
..   ...                                          ...                     ...   
350  360  Patrick and Beatrice Haggerty Museum of Art  marquette.edu/haggerty   
351  361                         Milwaukee Art Museum                 mam.org   
352  362         Villa Terrace Decorative Arts Museum        villaterrace.org   
353  363                      Museum of Wisconsin Art        wisconsinart.org   
354  364              National Museum of Wildlife Art         wildlifeart.org   

             City      Stat

# Calculate Index

In [275]:
# create new column dividing median income by 12 to find median monthly income
df['Monthly Median Income'] = df['Median Income'] / 12

# round Monthly Median Income to one decimal place
df['Monthly Median Income'] = df['Monthly Median Income'].round(1)
df



# display index


,ID,Museum,Website,City,State,ZIP Code,General Admission,Median Income,Monthly Median Income
0,1,Birmingham Museum of Art,artsbma.org,Birmingham,Alabama,35203,0.0,42596.0,3549.7
1,2,Huntsville Museum of Art,hsvmuseum.org,Huntsville,Alabama,35801,10.0,97643.0,8136.9
2,3,Mobile Museum of Art,mobilemuseumofart.com,Mobile,Alabama,36608,12.0,59400.0,4950.0
3,4,Montgomery Museum of Fine Arts,mmfa.org,Montgomery,Alabama,36117,0.0,73919.0,6159.9
4,5,Anchorage Museum of Art and History,anchoragemuseum.org,Anchorage,Alaska,99501,18.0,73540.0,6128.3
...,...,...,...,...,...,...,...,...,...
350,360,Patrick and Beatrice Haggerty Museum of Art,marquette.edu/haggerty,Milwaukee,Wisconsin,53211,0.0,73783.0,6148.6
351,361,Milwaukee Art Museum,mam.org,Milwaukee,Wisconsin,53202,19.0,62505.0,5208.8
352,362,Villa Terrace Decorative Arts Museum,villaterrace.org,Milwaukee,Wisconsin,53211,10.0,73783.0,6148.6
353,363,Museum of Wisconsin Art,wisconsinart.org,West Bend,Wisconsin,53095,15.0,77397.0,6449.8


In [276]:
# create Index column ny dividing general admission by monthly median income
df[['Index']] = df[['General Admission']].div(df['Monthly Median Income'], axis=0)

# multiply index by 10,000 to find whole number
df['Index'] = df ['Index'] * 10000

# round Index to one decimal place
df['Index'] = df['Index'].round(1)
df

,ID,Museum,Website,City,State,ZIP Code,General Admission,Median Income,Monthly Median Income,Index
0,1,Birmingham Museum of Art,artsbma.org,Birmingham,Alabama,35203,0.0,42596.0,3549.7,0.0
1,2,Huntsville Museum of Art,hsvmuseum.org,Huntsville,Alabama,35801,10.0,97643.0,8136.9,12.3
2,3,Mobile Museum of Art,mobilemuseumofart.com,Mobile,Alabama,36608,12.0,59400.0,4950.0,24.2
3,4,Montgomery Museum of Fine Arts,mmfa.org,Montgomery,Alabama,36117,0.0,73919.0,6159.9,0.0
4,5,Anchorage Museum of Art and History,anchoragemuseum.org,Anchorage,Alaska,99501,18.0,73540.0,6128.3,29.4
...,...,...,...,...,...,...,...,...,...,...
350,360,Patrick and Beatrice Haggerty Museum of Art,marquette.edu/haggerty,Milwaukee,Wisconsin,53211,0.0,73783.0,6148.6,0.0
351,361,Milwaukee Art Museum,mam.org,Milwaukee,Wisconsin,53202,19.0,62505.0,5208.8,36.5
352,362,Villa Terrace Decorative Arts Museum,villaterrace.org,Milwaukee,Wisconsin,53211,10.0,73783.0,6148.6,16.3
353,363,Museum of Wisconsin Art,wisconsinart.org,West Bend,Wisconsin,53095,15.0,77397.0,6449.8,23.3
